In [1]:
import tensorflow as tf 
import numpy as np 
import matplotlib as mp
mp.use('TkAgg')
import matplotlib.pyplot as plt 
import os as os
import pandas as pd
from ML_tools import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from pyspark.sql.types import DateType
from sklearn.preprocessing import StandardScaler

2023-08-10 14:50:10.171511: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 14:50:10.204310: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-10 14:50:10.204597: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-10 14:50:10.781482: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
spark = SparkSession.builder \
        .appName('Company_Project') \
        .config("spark.jars", "mariadb-java-client-3.1.4.jar")\
        .getOrCreate()



23/08/10 14:50:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/10 14:50:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:

DOCS = spark.read.format("jdbc")\
        .option("url","jdbc:mariadb://localhost:3306/lnd")\
        .option("driver", "org.mariadb.jdbc.Driver")\
        .option("dbtable", "DOCS")\
        .option("user", "ETL")\
        .option("password", os.environ.get('PASS'))\
        .load()

In [4]:
w = Window().partitionBy().orderBy(col('Date'))
DOCS_ML = DOCS.withColumn("PrevClose", lag("close", 1, 0).over(w)) \
        .withColumn("Return", (col("close") - col("PrevClose"))/ col("PrevClose")) \
        .withColumn("Date", DOCS.Date.cast(DateType()))
DOCS_pd = DOCS_ML.toPandas()

23/08/10 14:50:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/10 14:50:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/10 14:50:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/10 14:50:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/08/10 14:50:18 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


In [26]:
input_data = DOCS_pd[['Open', 'High', 'Low', 'Close', 'Volume']].values
targets =DOCS_pd['Return'].values



In [75]:
# Setup the data sets 

# N = number of samples in the data set
# T = Sequence length
# D =number of featurs
# M = number of hidden units
# K = number of output units


# T is the number of previous steps we will use to calculate the next 

# D is 1 as we are only using 1 feature in this case Return 
T= 10
D= input_data.shape[1]
N= input_data.shape[0]
X = []
Y = [] 

for t in range(len(input_data) - T):
    x = input_data[t:t+T]
    X.append(x)
    y = targets[t+T]
    Y.append(y)

X = np.array(X).reshape(-1, T, D) #now data should be N x T x D
Y = np.array(Y)
N = len(X)

In [76]:
print(X.shape, Y.shape)

(525, 10, 5) (525,)


In [77]:
x_train = X[:-N//2]
y_train = Y[:-N//2]
x_test = X[-N//2:]
y_test = Y[-N//2:]

In [83]:
# Create the Model 

i = tf.keras.layers.Input(shape=(T,D))
x = tf.keras.layers.LSTM(50)(i)
o = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(i, o)

model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam('learning_rate'==0.01))

In [85]:

r = model.fit(x_train, y_train, 
              epochs = 90,
              validation_data=(x_test, y_test),
              )

Epoch 1/90
9/9 [==============================] - 0s 10ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 2/90
9/9 [==============================] - 0s 6ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 3/90
9/9 [==============================] - 0s 7ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 4/90
9/9 [==============================] - 0s 6ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 5/90
9/9 [==============================] - 0s 6ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 6/90
9/9 [==============================] - 0s 6ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 7/90
9/9 [==============================] - 0s 5ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 8/90
9/9 [==============================] - 0s 5ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 9/90
9/9 [==============================] - 0s 6ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 10/90
9/9 [==============================] - 0s 6ms/step - loss: 0.0205 - val_loss: 0.0404
Epoch 11/90
9/9 [=====================

In [86]:
# multi step forcast 
validation_target = y_test
validation_predictions = []

# last train input 
last_x = x_test

while len (validation_predictions) < len(validation_target):
#     1x1 array -> scalar
    p = model.predict(last_x)[0,0]
    # update the prediction list
    validation_predictions.append(p)
    # make the new input 
    last_x = np.roll(last_x, -1)
    last_x[-1] = p 

9/9 [==============================] - 0s 877us/step


In [87]:
plt.plot(validation_target, label='forcast target')
plt.plot(validation_predictions, label='forcast prediction')
plt.legend()
plt.show()
        